In [50]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import os

`import_data`

De nagelopen corpustabel wordt geladen. Komma's worden omgezet in punten en opgeslagen als `float` zodat deze als geldige coördinaten kunnen worden gelezen.

In [51]:
def import_data(corpus):
    corpus_dataframe = pd.read_csv(corpus, sep='\t')
    corpus_dataframe['latitude'] = corpus_dataframe['latitude'].str.replace(',', '.').astype(float)
    corpus_dataframe['longitude'] = corpus_dataframe['longitude'].str.replace(',', '.').astype(float)
    corpus_dataframe = corpus_dataframe[corpus_dataframe['latitude'].notna()]
    
    return corpus_dataframe

`create_heatmap`

Er wordt een kaart gemaakt van de locaties die in het corpus worden genoemd. 

In [52]:
def create_map(current_dir, corpus_coordinates):
    latitude = corpus_coordinates['latitude'].mean()
    longitude = corpus_coordinates['longitude'].mean()
    
    heatmap_data = corpus_coordinates.groupby(['placename', 'latitude', 'longitude']).size().reset_index(name='count')
    map = folium.Map(location=[latitude, longitude], control_scale=True, zoom_start = 8)

    for index, row in heatmap_data.iterrows():
        folium.CircleMarker([row['latitude'], 
                             row['longitude']], 
                             radius = 2.5, 
                             color = 'black', 
                             fill_color = '#3AB795', 
                             fill = True, 
                             fill_opacity = 1).add_to(map)

    map.save(current_dir + '/output/corpus_locations.html')

`create_chronicle_heatmaps`

Er worden heat maps gemaakt op de kaart van de locaties die in het corpus worden genoemd. Het bestand `corpus_heatmap.html` bevat een heatmap voor de samengevoegde kronieken en de locaties die worden genoemd. Vervolgens wordt ook voor iedere unieke kroniek een heatmap gemaakt.

In [53]:
def create_chronicle_heatmaps(current_dir, corpus_coordinates):
    latitude = corpus_coordinates['latitude'].mean()
    longitude = corpus_coordinates['longitude'].mean()
    
    heatmap_data = corpus_coordinates.groupby(['placename', 'latitude', 'longitude']).size().reset_index(name='count')
    map = folium.Map(location = [latitude, longitude], control_scale = True, zoom_start=8)
    
    heat_data = [[row['latitude'], 
                  row['longitude'], 
                  row['count']] for index, row in heatmap_data.iterrows()]
    
    HeatMap(heat_data).add_to(map)
    map.save(current_dir + '/output/heatmaps/corpus_heatmap.html')

    for kroniek_value in corpus_coordinates['kroniek'].unique():
        filtered_dataframe = corpus_coordinates[corpus_coordinates['kroniek'] == kroniek_value]
        heatmap_data = filtered_dataframe.groupby(['placename', 'latitude', 'longitude']).size().reset_index(name='count')
        map = folium.Map(location = [latitude, longitude], zoom_start=8)
        heat_data = [[row['latitude'], 
                      row['longitude'], 
                      row['count']] for index, row in heatmap_data.iterrows()]
        
        HeatMap(heat_data).add_to(map)
        save_path = f'output/heatmaps/heatmap_{kroniek_value}.html'
        map.save(save_path)

In [54]:
def main():

    current_dir = os.getcwd()
    corpus = current_dir + '\\data\\checked_entity linking results.csv'

    corpus_coordinates = import_data(corpus)
    create_chronicle_heatmaps(current_dir, corpus_coordinates)
    create_map(current_dir, corpus_coordinates)

    return corpus_coordinates

In [55]:
if __name__ == '__main__':
    dataframe = main()